In [42]:
import boto3
import botocore
import glob
import imageio
import json
import marsican_functions as msn
import os
import pandas as pd
import pickle
import sys

from collections import Counter
from PIL import Image

if 'win' in sys.platform:
    print('On Windows')
    os.environ["AWS_SHARED_CREDENTIALS_FILE"] = 'C:\\Users\\acsch\\OneDrive\\Documents\\credentials.txt'
    
BUCKET = 'as-colony-count-images'

On Windows


In [3]:
s3 = boto3.resource('s3')

## Creating the bucket

In [7]:
# https://www.accadius.com/post-file-aws-s3-windows-python-3-program/

try:
    # The name must be unique across all existing bucket names in Amazon S3.
    s3.create_bucket(Bucket='as-colony-count-images')
    print("Created new bucket: {}".format('as_colony_count_images'))
except botocore.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'BucketAlreadyExists':
        print("Bucket exists already...no problem.")
    elif error_code == 'BucketAlreadyOwnedByYou':
        print("Bucket exists already...no problem.")
    else:
        print('Error code: ' + e.response['Error']['Code'])
        print(e)

Created new bucket: as_colony_count_images


## Find all relevant plate images

In [2]:
plate_metadata = glob.glob('F:\Colonies_Data\*\metadata.json')
all_img_paths = list()
for metadata_loc in plate_metadata:
    absolute_path = metadata_loc.split("\\metadata.json")[0]
    with open(metadata_loc, 'r') as f:
        colony_pic_metadata_json = f.read()
        colony_pic_metadata = json.loads(colony_pic_metadata_json)
    img_paths = msn.extract_paths_with_colonies(colony_pic_metadata['images_list'], max_colonies=99 )
    full_img_paths = [(path[0], absolute_path + '/' + path[1]) for path in img_paths]
    all_img_paths.append(full_img_paths)
all_img_paths = msn.flatten(all_img_paths)

Lt100_colonies_images = [t[1] for t in all_img_paths]

In [38]:
Lt100_colonies_images[300:400]

['F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/F/T4h_inf/IMG_Niguarda1_1578_9_F6_T722.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/S/T4h_inf/IMG_Niguarda1_1585_51_S54_T970.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/F/T4h_inf/IMG_Niguarda1_1716_9_F6_T542.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/S/T4h_inf/IMG_Niguarda1_1774_755_S54_T721.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/S/T4h_inf/IMG_Niguarda1_1830_9_S54_T962.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/S/T4h_inf/IMG_Niguarda1_1862_695_S54_T723.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/S/T4h_inf/IMG_Niguarda1_1870_53_S54_T733.png',
 'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part5/F/T4h_inf/IMG_Niguarda1_1880_755_F6_T721.png',
 'F

In [39]:
col_counts_rel_paths = [(t[0], t[1].split('inf/')[1]) for t in all_img_paths]

In [31]:
cc_rp_df = pd.DataFrame(col_counts_rel_paths, columns=['colony_count', 'relative_path'])
cc_rp_df_json = cc_rp_df.to_json('colony_count_relative_paths.json',orient='index')

In [23]:
counts_and_images = [(t[0], Image.fromarray(imageio.imread(t[1])).resize((1000,1000)).convert('L')) for t in all_img_paths]

KeyboardInterrupt: 

In [40]:
abs_rel_paths = [('images/' + t[1].split('inf/')[1], t[1]) for t in all_img_paths]
abs_rel_paths[:3]

[('images/IMG_Niguarda1_127_121_F6_T543.png',
  'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part9/F/T4h_inf/IMG_Niguarda1_127_121_F6_T543.png'),
 ('images/IMG_Niguarda1_1352_710_S54_T962.png',
  'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part9/S/T4h_inf/IMG_Niguarda1_1352_710_S54_T962.png'),
 ('images/IMG_Niguarda1_1421_610_F6_T541.png',
  'F:\\Colonies_Data\\MicrobIA_Dataset_only_counted_processed_anonimyzed_part9/F/T4h_inf/IMG_Niguarda1_1421_610_F6_T541.png')]

In [44]:
len(Counter([t[0] for t in abs_rel_paths]).keys())

1204

## Upload the files to S3

In [32]:
client = boto3.client('s3')

client.upload_file('colony_count_relative_paths.json', BUCKET, 'colony_count_relative_paths.json')

In [45]:
for ele in abs_rel_paths:
    client.upload_file(ele[1], BUCKET, ele[0])

KeyboardInterrupt: 